In [13]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression

In [14]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [15]:
entries = []
results = []
cleaner = F1DataCleaner(seasonsData, qualiResultsData, driversData, constructorsData, enginesData)
cleaner.constructDataset(entries, results)


In [16]:
#print(entries)

In [17]:
#print(results)

In [18]:
X = np.array(entries)
y = results
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.5708053845144507

In [19]:
reg.coef_

array([ 0.61836663,  0.67341912,  0.23987154, -0.03143205,  0.04131197,
        0.02883261])

In [20]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#print(newDrivers)
driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = cleaner.drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = cleaner.drivers[int(did)].constructor.name
with open('../F1PredictWeb/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [21]:
circuit = 32
circuitName = "Mexican"

with open('../F1PredictWeb/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [22]:
predictedEntrants = []

for did, cid in newDrivers.items():
    if circuit not in cleaner.drivers[did].trackpwr:
        cleaner.drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.trackpwr:
        cleaner.drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.engine.trackpwr:
        cleaner.drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        cleaner.drivers[did].pwr,
        cleaner.drivers[did].constructor.pwr, 
        cleaner.drivers[did].constructor.engine.pwr,
        cleaner.drivers[did].trackpwr[circuit],
        cleaner.drivers[did].constructor.trackpwr[circuit],
        cleaner.drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)
    

In [23]:
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[-0.98436805  0.5543021  -0.83821011 -0.84046643 -0.00459675 -0.00817045
  0.0938329  -0.56227476 -0.91022585  0.18923159  0.08133336 -0.60592381
  0.03019535  0.73196821  0.01964434  0.80432828  0.27918485  0.41780067
  0.78661654 -0.16661582]


In [24]:
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name, predictedResults[index]))

Lewis Hamilton (Mercedes): -0.9843680490254763
Fernando Alonso (McLaren): 0.5543020978881295
Kimi Räikkönen (Ferrari): -0.8382101090059912
Sebastian Vettel (Ferrari): -0.8404664257963677
Romain Grosjean (Haas F1 Team): -0.004596748765014015
Nico Hülkenberg (Renault): -0.008170448806282576
Sergio Pérez (Force India): 0.09383290005784645
Daniel Ricciardo (Red Bull): -0.5622747607738894
Valtteri Bottas (Mercedes): -0.9102258484605983
Kevin Magnussen (Haas F1 Team): 0.18923159316474775
Marcus Ericsson (Sauber): 0.08133335537197335
Max Verstappen (Red Bull): -0.6059238075075668
Carlos Sainz (Renault): 0.030195348870883768
Stoffel Vandoorne (McLaren): 0.7319682128461301
Esteban Ocon (Force India): 0.019644336814076697
Lance Stroll (Williams): 0.8043282810768612
Pierre Gasly (Toro Rosso): 0.2791848542406474
Brendon Hartley (Toro Rosso): 0.4178006745583349
Sergey Sirotkin (Williams): 0.7866165387903942
Charles Leclerc (Sauber): -0.16661582373357517
